### 下载qwen模型

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
cache_dir = "/raid/gfc/llm/models"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct",cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct",cache_dir=cache_dir).to(device)
print(model)
print('--------------------------------')
print(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.